# Multi-agentic orchestration using the Azure Open AI Chat Client

In [ ]:
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from agent_framework import WorkflowEvent, WorkflowBuilder, WorkflowOutputEvent

In [ ]:
# 📦 Import Configuration and Environment Management
import os                      # Environment variable access for secure configuration
from dotenv import load_dotenv # Secure loading of API credentials and settings

load_dotenv()                  # Load environment variables from .env file

In [ ]:
# 🎩 Agent 1: Hotel Concierge - Quality Reviewer Role
# This agent specializes in evaluating travel recommendations for authenticity
# Acts as the second stage in our multi-agent workflow for quality assurance
REVIEWER_NAME = "Concierge"
REVIEWER_INSTRUCTIONS = """
    You are an experienced hotel concierge who has strong opinions about providing the most local and authentic experiences for travelers.
    
    Your role in this multi-agent workflow:
    - Review travel recommendations from the Front Desk agent
    - Assess whether suggestions provide authentic, non-touristy experiences
    - Approve recommendations that meet high standards for local authenticity
    - Provide constructive feedback for refinement without giving specific examples
    
    Always focus on the quality and authenticity of experiences rather than just popular tourist destinations.
    """

In [ ]:
FRONTDESK_NAME = "FrontDesk"
FRONTDESK_INSTRUCTIONS = """
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
    """

In [ ]:
class DatabaseEvent(WorkflowEvent): ...

In [ ]:
aoai_chat_client = AzureOpenAIChatClient(credential=AzureCliCredential())

In [ ]:
reviewer_agent = aoai_chat_client.create_agent(
    instructions=REVIEWER_INSTRUCTIONS,
    name=REVIEWER_NAME
)

frontdesk_agent = aoai_chat_client.create_agent(
    instructions=FRONTDESK_INSTRUCTIONS,
    name=FRONTDESK_NAME
)

In [ ]:
workflow = WorkflowBuilder().set_start_executor(frontdesk_agent).add_edge(frontdesk_agent, reviewer_agent).build()

In [ ]:
from agent_framework import AgentRunEvent, WorkflowStatusEvent


status = await workflow.run("I would like to go to Rome.", include_status_events=True)
for event in status:
    if isinstance(event, WorkflowStatusEvent):
        print(f"WorkflowStatusEvent: {event.state}")
    if isinstance(event, AgentRunEvent):
        print(f"AgentRunEvent: {event.data}")
    else:
        print(f"Intermediate event: {event}")